Домашнее задание:
a. Сгенерировать меньший датасет из 8-10 классов движения
b. Обучить уже существующую модель (предварительно проанализировав
какие параметры модели нужно изменить)
c. Изменить модель: посмотреть зависимость от количества LSTM модулей
в нашей модели
d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии
и сравнить улучшилось или ухудшилось качество предсказания.
Провести несколько таких итераций, дать свою оценку уменьшению и
увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.
Желательно сделать так, чтобы длина последовательности
передавалась как атрибут класса.
2. Дополнительное задание:
a. http://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+c
onsumption - 2075259 measurements gathered in a house located in Sceaux
(7km of Paris, France) between December 2006 and November 2010 (47
months). Проделайте весь путь подготовки данных, создания датасета,
разделения и обучения модели самостоятельно. Предсказывать нужно
Global_active_power. Обратите внимание, что здесь задача регрессии, а
не классификации, т.е. модель нужно изменить.

### - a. Сгенерировать меньший датасет из 8-10 классов движения

In [86]:
import lib.util as myutils

In [14]:
import torch
from torch import nn
import numpy as np
import pandas as pd

In [87]:
data_path = "E:/ML/nturgbd_skeletons_s001_to_s017/nturgb+d_skeletons/"
#### список отсутсвующих элементов так же будет доступен 
broken_files_path = "E:/ML/NTU_RGBD_samples_with_missing_skeletons.txt"

In [107]:
training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = [8, 10, 21, 22, 23, 27, 30, 41, 42, 58] #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
training_cameras = [1, 2, 3] 

max_body_true = 1
max_body_kinect = 1

num_joint = 25
max_frame = 300

In [108]:
##### список файлов с лейблами на каждый файл 
working_files_with_labels, action_classes = myutils.read_data(data_path, 
broken_files_path, 
training_subjects, 
training_classes, 
training_cameras)

action classes:  {8: 0, 10: 1, 21: 2, 22: 3, 23: 4, 27: 5, 30: 6, 41: 7, 42: 8, 58: 9}
action files:  {8: 120, 10: 120, 21: 120, 22: 120, 23: 120, 27: 120, 30: 120, 41: 120, 42: 120, 58: 120}


In [100]:
LABELS = {v: k for k, v in action_classes.items()}

In [101]:
print(LABELS)

{0: 8, 1: 10, 2: 21, 3: 22, 4: 23, 5: 27, 6: 30, 7: 41, 8: 58}


In [92]:
len(working_files_with_labels)

1200

In [94]:
data = []
labels = []
##########################################################################
numbers = {0: 0, 1 : 0, 2 : 0, 3 : 0, 4 :0, 5 :0, 6:0, 7:0, 8:0, 9:0} #####
##################################################################
for file in working_files_with_labels:
    frames_blocks, label = myutils.create_coords_blocks(file, data_path)
    if label != [] and numbers[label[0]] <= 150:
        numbers[label[0]] = numbers[label[0]] + len(label)
        data = data + frames_blocks
        labels = labels + label
data_np = np.asarray(data)
labels_np = np.asarray(labels)

data_sq = data_np.reshape(len(data_np), -1)
test_data = pd.DataFrame(data_sq)
test_labels = pd.DataFrame(labels_np)
test_data['labels'] = test_labels

In [95]:
test_data.to_csv("skeletons_classes_1_10.csv", index = False)

### b. Обучить уже существующую модель (предварительно проанализировав какие параметры модели нужно изменить)

In [96]:
skeletons = pd.read_csv("skeletons_classes_1_10.csv")

In [97]:
skeletons.head()


,0,1,2,3,4,5,6,7,8,9,...,3366,3367,3368,3369,3370,3371,3372,3373,3374,labels
0,-0.080141,0.135073,3.691227,-0.058513,0.396866,3.606265,-0.037811,0.652906,3.509790,-0.066903,...,-0.066586,-0.194186,3.880914,0.103227,-0.183070,3.717531,0.180819,-0.130425,3.724300,0
1,-0.035358,0.192430,3.681518,-0.021036,0.422171,3.606487,-0.007186,0.646429,3.520689,-0.042399,...,-0.247820,0.496548,3.406900,-0.056666,0.495373,3.269386,-0.050549,0.490788,3.258667,1
2,0.430001,0.024285,4.098490,0.424111,0.227365,4.005508,0.417157,0.427353,3.902021,0.395301,...,0.098534,0.515498,3.738000,0.243639,0.441711,3.635301,0.292146,0.468907,3.595111,2
3,0.399282,-0.026338,4.099298,0.382165,0.177244,4.002089,0.359935,0.376536,3.893836,0.338453,...,0.300487,0.700893,3.694455,0.535627,0.814146,3.868758,0.530421,0.797046,3.864750,3
4,0.449151,0.001842,4.170505,0.412885,0.192257,4.103466,0.375976,0.379683,4.026225,0.401664,...,0.723439,0.436599,4.077281,0.535588,0.585105,4.202623,0.536817,0.559229,4.186821,3


In [55]:
skeletons.shape

(1365, 3376)

In [56]:
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms

In [57]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        self.labels = self.data.iloc[:,-1]
        self.transform = transform


    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
            
        

        return (item, label) 

In [59]:
dataset = Skeleton_Dataset(file_path = "skeletons_classes_1_10.csv", transform=None)

In [72]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.70*len(dataset)),int(0.30*len(dataset)+1)])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [74]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [76]:
device = torch.device("cpu")

In [77]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [78]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [82]:
from torch import optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.2993  / 10 ✗ (60)
epoch : 8 iter : 20 (2m 23s) 2.2886  / 60 ✗ (30)
epoch : 16 iter : 40 (4m 44s) 2.2774  / 8 ✓
epoch : 25 iter : 0 (7m 8s) 2.0064  / 22 ✗ (41)
epoch : 33 iter : 20 (9m 54s) 1.6172  / 30 ✓
epoch : 41 iter : 40 (12m 23s) 1.5743  / 22 ✓
epoch : 50 iter : 0 (14m 46s) 1.6090  / 8 ✗ (41)
epoch : 58 iter : 20 (17m 10s) 1.6460  / 27 ✓
epoch : 66 iter : 40 (19m 32s) 0.8845  / 58 ✓
epoch : 75 iter : 0 (21m 53s) 1.3206  / 41 ✗ (21)
epoch : 83 iter : 20 (24m 15s) 1.2362  / 30 ✗ (21)
epoch : 91 iter : 40 (26m 49s) 0.9656  / 22 ✓
epoch : 100 iter : 0 (29m 30s) 0.8203  / 21 ✗ (22)
epoch : 108 iter : 20 (32m 11s) 1.1220  / 8 ✓
epoch : 116 iter : 40 (34m 47s) 0.9514  / 10 ✓
epoch : 125 iter : 0 (37m 16s) 0.4492  / 30 ✓
epoch : 133 iter : 20 (39m 42s) 0.6063  / 30 ✗ (8)
epoch : 141 iter : 40 (42m 11s) 0.5200  / 60 ✓
epoch : 150 iter : 0 (44m 37s) 0.5867  / 22 ✓
epoch : 158 iter : 20 (47m 2s) 0.5163  / 30 ✓
epoch : 166 iter : 40 (49m 29s) 0.4766  / 22 ✗ (21)
e

In [83]:
%matplotlib notebook
plt.plot(list(range(0,len(all_losses))),all_losses)
plt.show()

<IPython.core.display.Javascript object>

In [85]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Accuracy of the network:   82.4390243902439


### c. Изменить модель: посмотреть зависимость от количества LSTM модулей в нашей модели

In [ ]:
class LSTM_net_ver2(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm1 = torch.nn.LSTM(input_dim, hidden_dim, layer_num,batch_first=True)
        self.dr1 = torch.nn.Dropout2d(0.1)
        self.lstm2 = torch.nn.LSTM(input_dim, hidden_dim)
        self.dr2 = torch.nn.Dropout2d(0.1)
        self.lstm3 = torch.nn.LSTM(input_dim, hidden_dim)
        self.dr3 = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out1,(hn,cn) = self.lstm1(x)
        lstm_out2,(hn,cn) = self.lstm2(x)
        lstm_out3,(hn,cn) = self.lstm3(x)
        out = self.fc(lstm_out3[:,-1,:])
        return out

### d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество. Желательно сделать так, чтобы длина последовательности передавалась как атрибут класса.